## Algorithm to reduce variables based on linear correlation and bivariate correlation 



### Requirements


1) Dataframe.

2) Clarity about response variable, it need to be a two level binary variable, and the format needs to be numeric.


In [4]:
#libraries
import pandas as pd
import scipy.stats as stats

In [1]:
#bivariate analysis function

#Dataframe: User needs to provide a dataset, format is a dataframe.
#bivary_Var: Name of the response variable column, string.


def biserial_correlation(dataframe, binary_var):
    num_df=dataframe._get_numeric_data()
    biser_corrs=[]
    colnames=[]
    df_columns=num_df.columns
    for i in range(0,len(num_df.columns)):
                   biserial_element=stats.pointbiserialr(num_df[df_columns[i]].dropna(), num_df[binary_var].dropna())
                   biser_corrs.insert(i+1,biserial_element[0])
                   colnames.insert(i+1,df_columns[i])
    return pd.DataFrame({"column":colnames,"corr":biser_corrs})

#Output is a dataframe

In [7]:
# find variables to remove based on linear correlation and bivariate correlation

# This algorithm select pair variables with a correlation above a certain threshold, from the pair we inspect their biserial correlation, the one with the lowest biserial correlation is a candidate for removal.


#Dataframe: User needs to provide a dataset, format is a dataframe.
#corr_thrsehold:  Correlations equal or above to this threshold will be revied, float number.
#target_var: Name of the response variable column, string.


def find_max_corr(dataframe,corr_threshold,target_var):
    df_corr=dataframe.corr().iloc[:,:len(dataframe.columns)]
    del[df_corr[target_var]]
    df_corr.drop(target_var, inplace=True)
    bis_corr=biserial_correlation(dataframe,'TARGET').iloc[:-1,:].set_index('column')
    to_remove=[]
    for i in range(0,len(df_corr)):
                   for j in range(i+1,len(df_corr)):
                       current_corr=df_corr.iloc[i,j]
                       if abs(current_corr)>=corr_threshold:
                                       removal_counter=0
                                       bis_1=abs(bis_corr.loc[df_corr.index[i]].values[0])
                                       bis_2=abs(bis_corr.loc[df_corr.index[j]].values[0])
                                       removal=df_corr.index[i] if bis_2>bis_1 else df_corr.index[j]
                                       to_remove.insert(i+1,removal)
                                       removal_counter+=1

    return to_remove


#Output is a list of varibles to be removed.

### Example

In [9]:

#import dataset
kggle=pd.read_csv('Kaggle_Train.csv',sep=';')

#Find list of variable to be removed, threshold is .8

variables=find_max_corr(kggle,0.8,'TARGET')    

#Some variable name may appear twice or more in the list, as they might have high correlation with more than one variable.

#Set statement will remove duplicates
total_removals=set(variables)

#We recreate the list
total_removals=list(total_removals)

#Removing the elemtns from original dataset.
for i in range(0,len(total_removals)):
               del[kggle[total_removals[i]]]